In [1]:
pip install pdfplumber camelot-py[cv] pytesseract opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install fpdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
print(os.getcwd())

C:\Users\Aryan


In [10]:
print(f"Saving cleaned text to {output_txt_path}")
print(f"Text content length: {len(cleaned_text)}")

Saving cleaned text to Cleaned_text_output.txt


NameError: name 'cleaned_text' is not defined

In [11]:
# Main function to run the extraction process and save to TXT
def main(pdf_path, output_txt_path):
    # Extract text from the PDF
    text_data = extract_text_from_pdf(pdf_path)
    print("Extracted Text:")
    print(text_data)

    # Extract tables from the PDF
    tables = extract_tables_from_pdf(pdf_path)
    for i, table in enumerate(tables):
        print(f"Table {i + 1} Data:")
        print(table)

    # Extract text from images (if necessary)
    image_text = extract_text_from_images(pdf_path)
    print("Extracted Text from Images:")
    print(image_text)

    # Combine all text (if you want to merge text and image-extracted text)
    combined_text = text_data + "\n" + image_text

    # Check the length of the combined text before saving
    print(f"Text content length: {len(combined_text)}")

    # Save cleaned text to a new TXT file
    save_text_to_txt(combined_text, output_txt_path)
    print(f"Cleaned text saved to {output_txt_path}")

    # Check if the file was created
    if os.path.exists(output_txt_path):
        print(f"File {output_txt_path} created successfully!")
    else:
        print(f"Failed to create {output_txt_path}")

# Specify your input PDF and output TXT paths here
pdf_path = "Test data.pdf"
output_txt_path = "Cleaned_text_output.txt"

# Run the process
main(pdf_path, output_txt_path)

Extracted Text:
Contents
Introduction                        
The growing relevance of social media . . . . . 
An Introduction to Media Monitoring . . . . . . . 
Areas of applicability for governments and
military organisations . . . . . . . . . . . . . . . . . . . 
Implementing the tools in daily workflow . . . . 
How to Assess the Social Media
Monitoring Tool                     
Using AI to assist with information
environment analysis . . . . . . . . . . . . . . . . . . . 
Methodology                      
Overview of Tools                  
Brandwatch . . . . . . . . . . . . . . . . . . . . . . . . . . 
Mediatoolkit . . . . . . . . . . . . . . . . . . . . . . . . . . 
Meltwater . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
BuzzSumo . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Keyhole . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Sprinklr . . . . . . . . . . . . . . .

DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.

In [9]:
import pdfplumber
import camelot
import pytesseract
import re
import os

# Function to clean extracted text, including removing page numbers and unwanted paragraph spacing
def clean_extracted_text(text):
    # Remove unwanted characters like continuous dots
    cleaned_text = re.sub(r'\.{2,}', '', text)  # Remove continuous dots
    cleaned_text = re.sub(r'\n+', '\n', cleaned_text)  # Remove extra newlines
    
    # Remove non-ASCII characters (handles strange characters like ����)
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', cleaned_text)  # Remove non-ASCII chars

    # Remove page numbers (assuming they appear in certain patterns, e.g., "Page 1", "1")
    cleaned_text = re.sub(r'Page \d+|\b\d+\b', '', cleaned_text)  # Remove "Page 1" or standalone digits

    # Remove unnecessary paragraph spacing (collapse multiple newlines to a single one)
    cleaned_text = re.sub(r'\n{2,}', '\n', cleaned_text)  # Collapse multiple newlines to a single one
    
    cleaned_text = cleaned_text.strip()  # Remove leading and trailing whitespace
    return cleaned_text

# Function to extract text using pdfplumber
def extract_text_from_pdf(pdf_path):
    text_data = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:  # Check if text extraction was successful
                text_data += text + "\n"
    return clean_extracted_text(text_data)  # Clean the extracted text

# Function to extract tables using camelot
def extract_tables_from_pdf(pdf_path):
    tables = camelot.read_pdf(pdf_path, pages='all')
    table_data = []
    for i, table in enumerate(tables):
        df = table.df
        table_data.append(df)
        df.to_csv(f'table_{i + 1}.csv', index=False)  # Save each table to CSV
    return table_data

# Function to preprocess image and perform OCR (if needed)
def extract_text_from_images(pdf_path):
    text_data = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Check if the page has images
            if page.images:
                for img in page.images:
                    x0, y0, x1, y1 = img['x0'], img['top'], img['x1'], img['bottom']
                    cropped_image = page.within_bbox((x0, y0, x1, y1)).to_image()
                    img_path = f'page_{page.page_number}.png'
                    cropped_image.save(img_path)
                    text = pytesseract.image_to_string(img_path)
                    text_data += text + "\n"
                    os.remove(img_path)  # Remove the temporary image
    return clean_extracted_text(text_data)  # Clean the extracted text from images

# Function to save text to a new TXT file
def save_text_to_txt(cleaned_text, output_txt_path):
    with open(output_txt_path, "w", encoding="utf-8") as text_file:
        text_file.write(cleaned_text)

# Main function to run the extraction process and save to TXT
def main(pdf_path, output_txt_path):
    # Extract text from the PDF
    text_data = extract_text_from_pdf(pdf_path)
    print("Extracted Text:")
    print(text_data)

    # Extract tables from the PDF
    tables = extract_tables_from_pdf(pdf_path)
    for i, table in enumerate(tables):
        print(f"Table {i + 1} Data:")
        print(table)

    # Extract text from images (if necessary)
    image_text = extract_text_from_images(pdf_path)
    print("Extracted Text from Images:")
    print(image_text)

    # Combine all text (if you want to merge text and image-extracted text)
    combined_text = text_data + "\n" + image_text

    # Save cleaned text to a new TXT file
    save_text_to_txt(combined_text, output_txt_path)
    print(f"Cleaned text saved to {output_txt_path}")

    # Check if the file was created
    if os.path.exists(output_txt_path):
        print(f"File {output_txt_path} created successfully!")
    else:
        print(f"Failed to create {output_txt_path}")

# Specify your input PDF and output TXT paths here
pdf_path = "Test data.pdf"
output_txt_path = "Cleaned_text_output.txt"

# Run the process
main(pdf_path, output_txt_path)

Extracted Text:
Contents
Introduction                        
The growing relevance of social media . . . . . 
An Introduction to Media Monitoring . . . . . . . 
Areas of applicability for governments and
military organisations . . . . . . . . . . . . . . . . . . . 
Implementing the tools in daily workflow . . . . 
How to Assess the Social Media
Monitoring Tool                     
Using AI to assist with information
environment analysis . . . . . . . . . . . . . . . . . . . 
Methodology                      
Overview of Tools                  
Brandwatch . . . . . . . . . . . . . . . . . . . . . . . . . . 
Mediatoolkit . . . . . . . . . . . . . . . . . . . . . . . . . . 
Meltwater . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
BuzzSumo . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Keyhole . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Sprinklr . . . . . . . . . . . . . . .

DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.